#### Observe that we can get news of a day with the link format:
http://news.ltn.com.tw/list/newspaper/politics/20181231
#### We need a list of date string in the date range we want.

In [4]:
#-*- coding:utf-8 -*-
from datetime import datetime, timedelta

start_date = "2019-02-28" # set start Date
stop_date = "2019-02-28" # set stop Date

start = datetime.strptime(start_date, "%Y-%m-%d")
stop = datetime.strptime(stop_date, "%Y-%m-%d")

dates = list()
while start <= stop:
    dates.append(start.strftime('%Y/%m/%d'))
    start = start + timedelta(days=1)


#### Now write a function to parse the HTML response, return the data we want(title, content, ...etc).

In [5]:
import requests
from bs4 import BeautifulSoup as bs

In [6]:
def process_document(document, date):
    newsBody = document.find(id='ranking_body')
    newsList = newsBody.select('table > tr')
    link = ''
    tag = ''
    title = ''

    data = list()
    
    for tr in newsList:
        td = tr.select('td')
        for element in td:
            if element.select_one('a') != None:
                # get link
                link = 'https://udn.com' + element.select_one('a')['href']
                # get title
                title = element.string
                # get content
                res = requests.get(link)
                doc = bs(res.text, 'lxml')
                body = doc.find(id='story_body_content')
                content = ""
                for ele in body.select('p'):
                    if ele.string != None:
                        content += ele.string
            elif ((len(element.get('class')) == 1) and (element.get('class')[0] == 'only_web')):
                # get tag
                tag = element.string
            else:
                continue
        if link != '':
            data.append({
            'date' : date,
            'title': title,
            'link' : link,
            'content' : content, 
            'tags' : tag
            })
        
    return data

#### Crawl over the news on the site, store the data in variable "all_data" .

In [7]:
cnt = 0
all_data = list()
for date in dates:
    print('start crawling :', date)
    res = requests.get('https://udn.com/news/archive/0/0/' + date)
    doc = bs(res.text, 'lxml')
    data = process_document(doc, date)
    all_data += data

start crawling : 2019/02/28


#### Check the result

In [8]:
all_data[0:5]

[{'date': '2019/02/28',
  'title': '不斷更新／破局！川普記者會：金正恩是好人 但我們決定這次不簽協定',
  'link': 'https://udn.com/news/story/12851/3669639',
  'content': '16:48川普在他下榻的萬豪酒店舉行的記者會，歷時約40分鐘，他在結束前說：「我即將搭機返程，回到華盛頓DC那個好地方。」BBC報導，川普顯然不想在河內多待一分鐘，16:40抵達機場，跟東道主越南官員道別，空軍一號在16:48起飛。15:59川普談及和金正恩關係，表示兩人的關係「非常溫暖」，並堅稱峰會並非在爭吵的緊繃狀況下結束，而是「一段非常友好的散步」。他回答媒體提問說：「這裡的走開不是像你站起來就走人那樣。我們的關係非常溫暖，我們的走開是非常友善的散步。」川普不忘批評歷任總統未對北韓問題採取行動，「這個問題早該在那麼多任總統期間解決，人們都在談論這個問題，卻沒做任何事。」15:40川普28日在記者會中表示，協議破局原因在制裁，「基本上他們希望全面解除對他們的制裁，但我們不能這麼做。」不過川普強調：「我們沒有放棄，他們有令人難以相信的、驚人的潛力。」表示他仍期待未來仍能舉行會談，「他（金正恩）想要去核，他想要先做的部分是我們覺得較不重要的。」川普回答記者提問時，承認峰會破局原因在美與北韓對解除制裁的意見不同。他說：「其實我非常想解除制裁，因為這個國家有那麼多的成長潛力。」川普也說，他會打電話給南韓總統文在寅，談談這次峰會，「等我上飛機後就會打電話給文總統。他一直很努力，他想見到協議敲定。」川普還說也會致電日本首相安倍晉三。15:27川金二會協議破局後，美國總統川普在28日下午2時多舉行記者會，開場他先談到印度和巴基斯坦衝突，強調正在調解，也提及委內瑞拉總統爭議。接著他進入正題談到峰會，表示：「這場會議很有建設性，金正恩是好人，是號人物。我們有些選項，但這次我們決定不做任何選擇。」川普說：「有時你必須走開，這次就是其中一次。」美國國務卿龐培歐在這項記者會中指出：「我們沒有獲得任何對美國而言有意義的事項，我認為金委員長對我們有期待，我們要求他做得更多，他還沒準備好。但我仍保持樂觀。」川普說，導致峰會破裂的原因是制裁問題，北韓要求全面解除制裁，美國不同意。14:43白宮發言人桑德斯28日發表聲明表